# Análisis de Clustering para Datos de Acciones de Samsung

Este notebook implementa un análisis de clustering utilizando K-means para identificar patrones en los datos de acciones de Samsung.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from datetime import datetime

# Configuración para visualizaciones
plt.style.use('seaborn')
sns.set(font_scale=1.2)
plt.rcParams['figure.figsize'] = (12, 8)

## Carga y Preparación de Datos

In [ ]:
# Cargar los datos
df = pd.read_csv('../data/samsung.csv')

# Convertir la columna de fecha a datetime
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# Exploración inicial
print("Información del dataset:")
df.info()

print("\nEstadísticas descriptivas:")
df.describe()

In [ ]:
# Visualizar la evolución temporal del precio de cierre
plt.figure(figsize=(14, 6))
plt.plot(df['Date'], df['Close'], marker='o', linestyle='-', alpha=0.7)
plt.title('Evolución del Precio de Cierre de Samsung')
plt.xlabel('Fecha')
plt.ylabel('Precio de Cierre')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Crear características adicionales
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

# Calcular características de tendencia
df['Close_Lag1'] = df['Close'].shift(1)
df['Volume_Lag1'] = df['Volume'].shift(1)
df['Close_Change'] = df['Close'] - df['Close_Lag1']
df['Volume_Change'] = df['Volume'] - df['Volume_Lag1']

# Eliminar filas con NaN (primera fila)
df = df.dropna()

# Seleccionar características para clustering
features = ['Close', 'Volume', 'Close_Change', 'Volume_Change']
X = df[features]

# Escalar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Guardar el scaler para uso futuro
import joblib
joblib.dump(scaler, '../models/scaler_samsung.pkl')

## Determinación del Número Óptimo de Clusters

In [ ]:
# Determinar el número óptimo de clusters usando el método del codo
inertia = []
silhouette_scores = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))

# Visualizar el método del codo
plt.figure(figsize=(12, 6))
plt.plot(k_range, inertia, marker='o', linestyle='-')
plt.xlabel('Número de Clusters')
plt.ylabel('Inercia')
plt.title('Método del Codo para Determinar el Número Óptimo de Clusters')
plt.grid(True)
plt.savefig('../results/samsung_elbow_method.png')
plt.show()

# Visualizar los puntajes de silueta
plt.figure(figsize=(12, 6))
plt.plot(k_range, silhouette_scores, marker='o', linestyle='-')
plt.xlabel('Número de Clusters')
plt.ylabel('Puntaje de Silueta')
plt.title('Puntaje de Silueta para Diferentes Números de Clusters')
plt.grid(True)
plt.show()

# Determinar el número óptimo de clusters
optimal_k = k_range[np.argmax(silhouette_scores)]
print(f"Número óptimo de clusters según el puntaje de silueta: {optimal_k}")

## Aplicación de K-means con el Número Óptimo de Clusters

In [ ]:
# Aplicar K-means con el número óptimo de clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df['Cluster'] = kmeans.fit_predict(X_scaled)

# Guardar el modelo K-means
joblib.dump(kmeans, '../models/modelo_samsung_kmeans.pkl')

# Guardar el dataset con los clusters asignados
df.to_csv('../results/samsung_con_clusters.csv', index=False)

## Visualización y Análisis de Clusters

In [ ]:
# Visualizar los clusters en las características originales
plt.figure(figsize=(12, 8))
sns.scatterplot(x='Close', y='Volume', hue='Cluster', data=df, palette='viridis', s=100, alpha=0.7)
plt.title('Clusters de Acciones de Samsung: Precio de Cierre vs Volumen')
plt.xlabel('Precio de Cierre')
plt.ylabel('Volumen')
plt.grid(True)
plt.show()

In [ ]:
# Aplicar PCA para visualizar los clusters en 2D
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Crear un DataFrame con los resultados de PCA
pca_df = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2'])
pca_df['Cluster'] = df['Cluster']

# Visualizar los clusters en el espacio PCA
plt.figure(figsize=(12, 8))
sns.scatterplot(x='PC1', y='PC2', hue='Cluster', data=pca_df, palette='viridis', s=100, alpha=0.7)
plt.title('Visualización de Clusters de Samsung en Espacio PCA')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.grid(True)
plt.savefig('../results/samsung_clusters_pca.png')
plt.show()

In [ ]:
# Visualizar la evolución temporal de los clusters
plt.figure(figsize=(14, 8))
sns.scatterplot(x='Date', y='Close', hue='Cluster', data=df, palette='viridis', s=100, alpha=0.7)
plt.title('Evolución Temporal de Clusters de Samsung')
plt.xlabel('Fecha')
plt.ylabel('Precio de Cierre')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../results/samsung_temporal_clusters.png')
plt.show()

In [ ]:
# Analizar las características de cada cluster
cluster_stats = df.groupby('Cluster')[features].mean()
print("Estadísticas promedio por cluster:")
print(cluster_stats)

# Visualizar el precio de cierre promedio por cluster
plt.figure(figsize=(10, 6))
cluster_stats['Close'].plot(kind='bar', color='skyblue')
plt.title('Precio de Cierre Promedio por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Precio de Cierre Promedio')
plt.grid(True, axis='y')
plt.savefig('../results/samsung_close_by_cluster.png')
plt.show()

## Interpretación de Resultados

- El análisis de clustering con K-means identificó [optimal_k] patrones distintos en los datos de acciones de Samsung.
- Cluster 1: [Descripción basada en los resultados, por ejemplo, 'Períodos de alto precio y bajo volumen']
- Cluster 2: [Descripción]
- Cluster 3: [Descripción]
- La visualización temporal muestra que los clusters tienden a agruparse en períodos específicos, lo que sugiere cambios en el comportamiento del mercado a lo largo del tiempo.
- Los resultados del clustering pueden ser útiles para estrategias de inversión, identificando patrones de comportamiento del mercado que podrían repetirse en el futuro.